In [1]:
from tables import *

C:\Users\spatz\Documents\Python\Jace 1.1\tables.py:310: RuntimeWarning: overflow encountered in ulonglong_scalars
  magic_index = (occupancy * bishop_magic_numbers[sq]) >> np.ulonglong(64 - bishop_relevant_bits[sq])
C:\Users\spatz\Documents\Python\Jace 1.1\tables.py:315: RuntimeWarning: overflow encountered in ulonglong_scalars
  magic_index = int((occupancy * rook_magic_numbers[sq]) >> np.ulonglong(64 - rook_relevant_bits[sq]))


In [8]:
import numpy as np
from bitboard_utils import print_bb, set_bit
from constants import algebraic_square_map

occ = np.ulonglong(0)
occ = set_bit(occ, algebraic_square_map["d4"])
occ = set_bit(occ, algebraic_square_map["d6"])
occ = set_bit(occ, algebraic_square_map["e3"])
occ = set_bit(occ, algebraic_square_map["f7"])
occ = set_bit(occ, algebraic_square_map["b7"])
occ = set_bit(occ, algebraic_square_map["b3"])
occ = set_bit(occ, algebraic_square_map["g2"])
occ = set_bit(occ, algebraic_square_map["b5"])
occ = set_bit(occ, algebraic_square_map["c5"])
occ = set_bit(occ, algebraic_square_map["e5"])

print_bb(occ)
print_bb(get_bishop_attacks(algebraic_square_map["d5"], occ))
print_bb(get_rook_attacks(algebraic_square_map["d5"], occ))



8  ·  ·  ·  ·  ·  ·  ·  · 
7  ·  1  ·  ·  ·  1  ·  · 
6  ·  ·  ·  1  ·  ·  ·  · 
5  ·  1  1  ·  1  ·  ·  · 
4  ·  ·  ·  1  ·  ·  ·  · 
3  ·  1  ·  ·  1  ·  ·  · 
2  ·  ·  ·  ·  ·  ·  1  · 
1  ·  ·  ·  ·  ·  ·  ·  · 
   A  B  C  D  E  F  G  H
Bitboard: 18034224448152064


8  ·  ·  ·  ·  ·  ·  ·  · 
7  ·  1  ·  ·  ·  1  ·  · 
6  ·  ·  1  ·  1  ·  ·  · 
5  ·  ·  ·  ·  ·  ·  ·  · 
4  ·  ·  1  ·  1  ·  ·  · 
3  ·  1  ·  ·  ·  1  ·  · 
2  ·  ·  ·  ·  ·  ·  1  · 
1  ·  ·  ·  ·  ·  ·  ·  · 
   A  B  C  D  E  F  G  H
Bitboard: 18051867805491712


8  ·  ·  ·  ·  ·  ·  ·  · 
7  ·  ·  ·  ·  ·  ·  ·  · 
6  ·  ·  ·  1  ·  ·  ·  · 
5  ·  ·  1  ·  1  ·  ·  · 
4  ·  ·  ·  1  ·  ·  ·  · 
3  ·  ·  ·  ·  ·  ·  ·  · 
2  ·  ·  ·  ·  ·  ·  ·  · 
1  ·  ·  ·  ·  ·  ·  ·  · 
   A  B  C  D  E  F  G  H
Bitboard: 34695806976


In [ ]:
from chess import *

def print_bb(bb):
    print("\n")
    for rank in range(8):
        r = ""
        for file in range(8):
            sq = rank * 8 + file
            r += f" {'1' if get_bit(bb, sq) else '·'} "
        print(8 - rank, r)
    print("   A  B  C  D  E  F  G  H")

    print("Bitboard:", bb)

In [ ]:
board = np.ulonglong(456)
print_bb(board)
print(count_bits(board))
print(get_lsb1_index(board))

In [ ]:
# sliders
bishop_attacks = init_sliders(np.empty((64, 512), dtype=np.ulonglong), bish=True)
rook_attacks = init_sliders(np.empty((64, 4096), dtype=np.ulonglong), bish=False)

In [ ]:
def get_bishop_attacks(sq, occ):
    if sq == 63:
        return bishop_attacks_on_the_fly(sq, occ)
    occ &= bishop_masks[sq]
    print(occ)
    print(bishop_magic_numbers[sq])
    occ *= bishop_magic_numbers[sq]
    occ >>= np.ulonglong(64 - bishop_relevant_bits[sq])
    return bishop_attacks[sq][occ]

def get_rook_attacks(sq, occ):
    occ &= rook_masks[sq]
    occ *= rook_magic_numbers[sq]
    occ >>= np.ulonglong(64 - rook_relevant_bits[sq])
    return rook_attacks[sq][occ]

In [ ]:
occ = np.ulonglong(784**5)
print_bb(occ)
print_bb(get_bishop_attacks(62, occ))
print_bb(get_rook_attacks(46, occ))